<a href="https://colab.research.google.com/github/nehagovekar/GEN-AI-Projects/blob/main/HelloAgents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time

class Message:
    """Simple message for agent communication"""
    def __init__(self, sender, receiver, content):
        self.sender = sender
        self.receiver = receiver
        self.content = content
        self.timestamp = time.time()

    def __str__(self):
        return f"[{self.sender} → {self.receiver}]: {self.content}"





In [2]:
class Agent:
    """Basic agent that can send and receive messages"""
    def __init__(self, name):
        self.name = name
        self.inbox = []
        self.outbox = []

    def receive_message(self, message):
        """Add a message to this agent's inbox"""
        self.inbox.append(message)
        print(f"{self.name} received: {message}")

    def send_message(self, receiver, content):
        """Create a message and add it to outbox"""
        message = Message(self.name, receiver, content)
        self.outbox.append(message)
        print(f"{self.name} sent: {message}")

    def process_messages(self):
        """Process all messages in the inbox"""
        for message in self.inbox:
            self.handle_message(message)
        self.inbox = []

    def handle_message(self, message):
        """Each agent type implements its own message handling"""
        pass

    def take_action(self):
        """Each agent type implements its own actions"""
        pass

    def step(self):
        """Run one cycle of the agent's behavior"""
        self.process_messages()
        self.take_action()



In [3]:
class HelloAgent(Agent):
    """An agent that responds to greetings"""
    def __init__(self, name):
        super().__init__(name)
        # Keep track of who we've already greeted
        self.greeted = set()

    def handle_message(self, message):
        # Check if this is a greeting message
        if "hello" in message.content.lower() or "hi" in message.content.lower():
            # Send a reply if we haven't already greeted this agent
            if message.sender not in self.greeted:
                self.send_message(message.sender, f"Hello, {message.sender}! Nice to meet you!")
                self.greeted.add(message.sender)

        # If someone asks a question, try to answer
        elif "?" in message.content:
            self.send_message(
                message.sender,
                f"You asked: '{message.content}' I'm just a simple agent, so I don't know the answer."
            )


class CuriousAgent(Agent):
    """An agent that asks questions"""
    def __init__(self, name):
        super().__init__(name)
        self.questions = [
            "What is your name?",
            "How do you work?",
            "What can you do?",
            "Are you an agent too?"
        ]
        self.question_index = 0
        self.asked = set()  # Keep track of which agents we've questioned

    def take_action(self):
        # Periodically ask questions to other agents
        if self.question_index < len(self.questions):
            # Find an agent we haven't questioned yet
            for agent_name in ["Alice", "Bob", "Charlie"]:
                if agent_name != self.name and agent_name not in self.asked:
                    question = self.questions[self.question_index]
                    self.send_message(agent_name, question)
                    self.asked.add(agent_name)
                    self.question_index += 1
                    break




In [5]:
class MultiAgentSystem:
    """Simple environment for agents to interact"""
    def __init__(self):
        # Create a few agents
        self.agents = {
            "Alice": HelloAgent("Alice"),
            "Bob": HelloAgent("Bob"),
            "Charlie": CuriousAgent("Charlie")
        }

    def run_step(self):
        """Run one step of the simulation"""
        # Each agent takes a turn
        for name, agent in self.agents.items():
            print(f"\n{name}'s turn:")
            agent.step()

        # Route messages
        self.deliver_messages()

    def deliver_messages(self):
        """Deliver messages from outboxes to inboxes"""
        print("\nDelivering messages:")
        for sender_name, sender in self.agents.items():
            for message in sender.outbox:
                if message.receiver in self.agents:
                    # Deliver the message
                    receiver = self.agents[message.receiver]
                    receiver.receive_message(message)
                else:
                    print(f"Error: Cannot deliver message to unknown agent '{message.receiver}'")

            # Clear the outbox
            sender.outbox = []

    def run_simulation(self, steps=5):
        """Run the simulation for a number of steps"""
        print("Starting simulation...\n")

        # Initialize with some greeting messages
        self.agents["Alice"].send_message("Bob", "Hello, Bob!")
        self.agents["Bob"].send_message("Charlie", "Hi Charlie, welcome!")

        # Run the simulation steps
        for i in range(steps):
            print(f"\n----- Step {i+1} -----")
            self.run_step()

        print("\nSimulation complete!")


# Run the simulation
if __name__ == "__main__":
    system = MultiAgentSystem()
    system.run_simulation()

Starting simulation...

Alice sent: [Alice → Bob]: Hello, Bob!
Bob sent: [Bob → Charlie]: Hi Charlie, welcome!

----- Step 1 -----

Alice's turn:

Bob's turn:

Charlie's turn:
Charlie sent: [Charlie → Alice]: What is your name?

Delivering messages:
Bob received: [Alice → Bob]: Hello, Bob!
Charlie received: [Bob → Charlie]: Hi Charlie, welcome!
Alice received: [Charlie → Alice]: What is your name?

----- Step 2 -----

Alice's turn:
Alice sent: [Alice → Charlie]: You asked: 'What is your name?' I'm just a simple agent, so I don't know the answer.

Bob's turn:
Bob sent: [Bob → Alice]: Hello, Alice! Nice to meet you!

Charlie's turn:
Charlie sent: [Charlie → Bob]: How do you work?

Delivering messages:
Charlie received: [Alice → Charlie]: You asked: 'What is your name?' I'm just a simple agent, so I don't know the answer.
Alice received: [Bob → Alice]: Hello, Alice! Nice to meet you!
Bob received: [Charlie → Bob]: How do you work?

----- Step 3 -----

Alice's turn:
Alice sent: [Alice → Bo